# Food Review Tone Analysis and Response
In this tutorial, we use EvaDB + ChatGPT to analyze whether a food review is negative or not. Based on the analysis, we then use EvaDB + ChatGPT again to form a polite response to address negative review.

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/staging/tutorials/14-food-review-tone-analysis-and-response.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/staging/tutorials/14-food-review-tone-analysis-and-response.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/raw/staging/tutorials/14-food-review-tone-analysis-and-response.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table><br><br>

## Start Postgres

In [1]:
!apt install postgresql
!service postgresql start

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl
  logrotate netbase postgresql-14 postgresql-client-14
  postgresql-client-common postgresql-common ssl-cert sysstat
Suggested packages:
  bsd-mailx | mailx postgresql-doc postgresql-doc-14 isag
The following NEW packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl
  logrotate netbase postgresql postgresql-14 postgresql-client-14
  postgresql-client-common postgresql-common ssl-cert sysstat
0 upgraded, 13 newly installed, 0 to remove and 16 not upgraded.
Need to get 18.3 MB of archives.
After this operation, 51.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 logrotate amd64 3.19.0-1ubuntu1.1 [54.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main am

## Create User and Database

In [2]:
!sudo -u postgres psql -c "CREATE USER eva WITH SUPERUSER PASSWORD 'password'"
!sudo -u postgres psql -c "CREATE DATABASE evadb"

CREATE ROLE
CREATE DATABASE


## Install EvaDB

In [18]:
%pip install --quiet "evadb[document]"
%pip install psycopg2

import evadb
cursor = evadb.connect().cursor()

In [20]:
import warnings
warnings.filterwarnings("ignore")

from IPython.core.display import display, HTML
def pretty_print(df):
    return display(HTML( df.to_html().replace("\\n","<br>")))

## Create Data Source in EvaDB
We use data source to connect EvaDB directly to underlying database systems like Postgres.

In [4]:
params = {
    "user": "eva",
    "password": "password",
    "host": "localhost",
    "port": "5432",
    "database": "evadb",
}
query = f"CREATE DATABASE postgres_data WITH ENGINE = 'postgres', PARAMETERS = {params};"
cursor.query(query).df()

,0
0,The database postgres_data has been successful...


## Create Review Table

In [5]:
cursor.query("""
USE postgres_data {
  DROP TABLE IF EXISTS review_table
}
""").df()

,status
0,success


In [6]:
cursor.query("""
USE postgres_data {
  CREATE TABLE review_table (name VARCHAR(10), review VARCHAR(1000))
}
""").df()

,status
0,success


## Insert Reviews into Postgres
In this example, we directly insert data into the table for simplicity. We can also load data from files like CSV using Postgres APIs.

In [8]:
query = """
USE postgres_data {
  INSERT INTO review_table (name, review) VALUES ('Customer 1', 'I ordered fried rice but it is too salty.')
}
"""
cursor.query(query).df()

,status
0,success


In [9]:
query = """
USE postgres_data {
  INSERT INTO review_table (name, review) VALUES ('Customer 2', 'I ordered burger. It tastes very good and the service is exceptional.')
}
"""
cursor.query(query).df()

,status
0,success


In [10]:
query = """
USE postgres_data {
  INSERT INTO review_table (name, review) VALUES ('Customer 3', 'I ordered a takeout order, but the chicken sandwidth is missing from the order.')
}
"""
cursor.query(query).df()

,status
0,success


## Review Table Content
Now we have 3 reviews from different customers stored in the table.

In [11]:
cursor.query("SELECT * FROM postgres_data.review_table;").df()

,review_table.name,review_table.review
0,Customer 1,ordered fried rice but it is too salty.
1,Customer 2,I ordered burger. It tastes very good and the ...
2,Customer 3,"I ordered a takeout order, but the chicken san..."


## Register OpenAI Token

In [14]:
import os
os.environ["OPENAI_KEY"] = "sk-..."

## Tone Analysis for All Reviews
Here, we use ChatGPT with customized prompt to summarize whether the review tone is "postive" or "negative".

In [15]:
cursor.query("""
SELECT ChatGPT(
  "Is the review positive or negative. Only reply 'positive' or 'negative'. Here are examples. The food is very bad: negative. The food is very good: postive.",
  review
)
FROM postgres_data.review_table;
""").df()

,chatgpt.response
0,negative
1,positive
2,negative


## Respond to Negative Reviews
EvaDB allows users to filter data based on function output. In this query, we construct a query to filter out "postive" reviews. We then use a second ChatGPT with customized prompt to propose a solution to address customer's negative reviews politely.

In [21]:
response_df = cursor.query("""
SELECT ChatGPT(
  "Respond the the review with solution to address the review's concern",
  review
)
FROM postgres_data.review_table
WHERE ChatGPT(
  "Is the review positive or negative. Only reply 'positive' or 'negative'. Here are examples. The food is very bad: negative. The food is very good: postive.",
  review
) = "negative";
""").df()

pretty_print(response_df)

,chatgpt.response
0,"Dear valued customer,Thank you for bringing this matter to our attention. We apologize for the inconvenience caused by the excessive saltiness of your fried rice. We understand how important it is to have a satisfying dining experience, and we would like to make it right for you.To address your concern, we have taken the following steps:1. Recipe adjustment: We have reviewed our fried rice recipe and made necessary adjustments to ensure that the saltiness is balanced and meets our customers' expectations.2. Staff training: We have conducted additional training sessions with our kitchen staff to emphasize the importance of proper seasoning and taste testing before serving any dish.3. Quality control: Our management team will be implementing stricter quality control measures to ensure that every dish leaving our kitchen meets our high standards.We would like to invite you to give us another chance to serve you. Please reach out to our customer service team, and we will be more than happy to offer you a replacement dish or a refund for your order. We value your feedback and want to ensure that you have a positive experience with us.Once again, we apologize for any inconvenience caused, and we appreciate your understanding. We look forward to the opportunity to make it right for you.Best regards,[Your Name][Restaurant Name]"
1,"Dear [Customer's Name],Thank you for bringing this issue to our attention. We apologize for the inconvenience caused by the missing chicken sandwich in your takeout order. We understand how frustrating it can be when an item is missing from your meal.To address this concern, we would like to offer you two possible solutions:1. Replacement: We can arrange for a new chicken sandwich to be prepared and delivered to your location as soon as possible. Please let us know your preferred time for the replacement.2. Refund: If you prefer not to receive a replacement, we can issue a refund for the missing chicken sandwich. The refund will be processed through the same payment method used for the original order.Please let us know which option you would prefer, and we will take immediate action to resolve this issue for you. We value your satisfaction and want to ensure that you have a positive experience with our service.Once again, we apologize for any inconvenience caused, and we appreciate your understanding. If you have any further questions or concerns, please don't hesitate to reach out to us.Best regards,[Your Name][Restaurant Name]"
